In [112]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [113]:
DATA_DIR = 'C:\\Users\\xxxxx\\Desktop\\7506\\2021\\tp\data\\'


In [114]:
#train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
#train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')

df_train = pd.read_csv(DATA_DIR+'train_values.csv', index_col='building_id')
df_train_labels = pd.read_csv(DATA_DIR+'train_labels.csv', index_col='building_id')

In [115]:
#from collections import Counter
#counter = Counter(df_train_labels)
#for label, _ in counter.items():
#    row_ix = where(y == label)[0]
#    pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
#pyplot.legend()
#pyplot.show()

In [116]:
import os
cwd = os.getcwd()

In [117]:
#selected_features = ['foundation_type', 
#                     'area_percentage', 
#                     'height_percentage',
#                     'count_floors_pre_eq',
#                     'land_surface_condition',
#                     'has_superstructure_cement_mortar_stone']

selected_features = [
'geo_level_3_id',
'geo_level_2_id',
'geo_level_1_id',
'area_percentage',
'age',
'height_percentage'
#'count_families',
#'count_floors_pre_eq',
#'has_superstructure_mud_mortar_stone',
#'has_superstructure_timber'
]

train_values_subset = df_train#[selected_features]
#train_values_subset = df_train

In [118]:
train_values_subset = pd.get_dummies(train_values_subset)

In [119]:
#Outliers
from scipy.stats.mstats import winsorize
#train_values_subset["age"] = winsorize(train_values_subset["age"],(0, 0.05))
#train_values_subset["area_percentage"] = winsorize(train_values_subset["area_percentage"],(0, 0.055))
#train_values_subset["height_percentage"] = winsorize(train_values_subset["height_percentage"],(0, 0.04))

In [120]:
train_values_subset

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,1,1,0,...,0,0,0,0,0,0,0,0,1,0
28830,8,900,2812,2,10,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
94947,21,363,8973,2,10,5,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
590882,22,418,10694,2,10,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
201944,11,131,1488,3,30,8,9,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,0,1,0,...,0,0,0,1,0,0,0,0,1,0
669485,17,715,2060,2,0,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
602512,17,51,8163,3,55,6,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [121]:
# for preprocessing the data
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

# the model
#from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier


# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [122]:
pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021))  

In [123]:
##param_test1 = {
## 'max_depth':range(3,10,2),
## 'min_child_weight':range(1,6,2)
##}

#gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
# min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
# objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
# param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
#gsearch1.fit(train[predictors],train[target])
#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [124]:

# Param grid for Xgboost
#param_grid = {
# 'classifier__max_depth':[1,2,3,4,5,6,7,8,9,10], # just as example
# 'classifier__n_estimators': 100
#}

param_grid = {
              'xgbclassifier__classifier__n_estimators': [50, 100,150,200],
              'xgbclassifier__classifier__max_depth': [3, 5,7, 10],
              'xgbclassifier__classifier__gamma': [0, 0.5, 1],
              'xgbclassifier__classifier__subsample': [0.6, 1.0, 1.2],
              'xgbclassifier__classifier__min_child_weight':[1,6,7,10]
             }

({'xgbclassifier__n_estimators': 200,
  'xgbclassifier__min_child_weight': 10,
  'xgbclassifier__max_depth': 10,
  'xgbclassifier__gamma': 1},)
#param_grid = {
#        'xgbclassifier__n_estimators': [273],    
#        'xgbclassifier__learning_rate': [0.03, 0.01, 0.003, 0.001],
#        'xgbclassifier__min_child_weight': [1,3, 5,7, 10],
#        'xgbclassifier__gamma': [0, 0.5, 1, 1.5, 2, 2.5, 5],
#        'xgbclassifier__subsample': [0.6, 0.8, 1.0, 1.2, 1.4],
#        'xgbclassifier__colsample_bytree': [0.6, 0.8, 1.0, 1.2, 1.4],
#        'xgbclassifier__max_depth': [3, 4, 5, 6, 7, 8, 9 ,10, 12, 14],
#        'xgbclassifier__reg_lambda':np.array([0.4, 0.6, 0.8, 1, 1.2, 1.4])}


#param_grid = {
#        'xgbclassifier__learning_rate': [0.03, 0.01],
#        'xgbclassifier__min_child_weight': [1,3,10],
#        'xgbclassifier__gamma': [0, 0.5, 1, 1.5],
#        'xgbclassifier__subsample': [0.6, 0.8, 1.0],
#        'xgbclassifier__colsample_bytree': [0.6, 0.8, 1.0],
#        'xgbclassifier__max_depth': [3, 4, 5, 6, 7],
#        'xgbclassifier__reg_lambda':np.array([0.4, 0.6, 1])}

# specific parameters. I set early stopping to avoid overfitting and specify the validation dataset 
fit_params = { 
        'xgbclassifier__early_stopping_rounds':10,
        'xgbclassifier__eval_set':[(train_values_subset, df_train_labels)]
         }

#gs = RandomizedSearchCV(pipe, param_grid, n_iter=1000, scoring="f1_micro", verbose=~1, cv=5)
gs = RandomizedSearchCV(pipe, param_grid, scoring="f1_micro", verbose=1, cv=5,n_jobs=-1)
#random_search = RandomizedSearchCV(xgb_pipeline, param_distributions=params, n_iter=1000,
#                                   scoring="precision", n_jobs=-1,  verbose=3, random_state=42, cv=3 )

In [125]:
#gs.fit(train_values_subset, df_train_labels.values.ravel(),**fit_params)
gs.fit(train_values_subset, df_train_labels.values.ravel())


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 19.4min finished
C:\Users\caste\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:59:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { classifier__gamma, classifier__max_depth, classifier__min_child_weight, classifier__n_estimators, classifier__subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:59:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('xgbclassifier',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            gamma=None,
                                                            gpu_id=None,
                                                            importance_type='gain',
                                                            interaction_constraints=None,
                                                            lea

In [126]:
#gs.grid_scores_

In [127]:
gs.best_params_,

({'xgbclassifier__classifier__subsample': 0.6,
  'xgbclassifier__classifier__n_estimators': 100,
  'xgbclassifier__classifier__min_child_weight': 7,
  'xgbclassifier__classifier__max_depth': 5,
  'xgbclassifier__classifier__gamma': 0.5},)

In [128]:
({'xgbclassifier__n_estimators': 200,
  'xgbclassifier__min_child_weight': 10,
  'xgbclassifier__max_depth': 10,
  'xgbclassifier__gamma': 1},)

({'xgbclassifier__n_estimators': 200,
  'xgbclassifier__min_child_weight': 10,
  'xgbclassifier__max_depth': 10,
  'xgbclassifier__gamma': 1},)

In [129]:
 gs.best_score_
#    0.7429825605195232


0.7272880809712767

In [130]:
from sklearn.metrics import f1_score

in_sample_preds = gs.predict(train_values_subset)
f1_score(df_train_labels, in_sample_preds, average='micro')
#0.8076062639821029
#0.8013246303736363
#0.8550197428252386

0.7452197036849436

In [109]:
test_values = pd.read_csv(DATA_DIR+'test_values.csv', index_col='building_id')


In [110]:
test_values_subset = test_values#[selected_features]
test_values_subset = pd.get_dummies(test_values_subset)

In [111]:
predictions = gs.predict(test_values_subset)


In [ ]:
submission_format = pd.read_csv(DATA_DIR+'submission_format.csv', index_col='building_id')


In [ ]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv(DATA_DIR+'submissionXGB_02.csv')